In [1]:
import numpy as np
import pescador
import logging
import os

import matplotlib
import matplotlib.pyplot as plt
import numpy as np

import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.cuda.amp import autocast
from torch.distributions.gamma import Gamma
from torch.distributions.normal import Normal

from IPython.display import display, clear_output

import math

import gc
import sys

from datetime import datetime

from functools import reduce

In [2]:
LOGGER = logging.getLogger('gbsd')
LOGGER.setLevel(logging.DEBUG)

In [3]:
torch.set_printoptions(sci_mode=False)
np.set_printoptions(suppress=True)

In [4]:
matplotlib.use('Agg')

In [5]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
    
cpu = torch.device('cpu')

In [6]:
CMD_VOLENVPER = 0
CMD_DUTYLL = 1
CMD_MSB = 2
CMD_LSB = 3
CMD_COUNT = 4

TIME_OFFSET = 0
CH_OFFSET = 1
CMD_OFFSET = 2
PARAM1_OFFSET = 3
PARAM2_OFFSET = 4
PARAM3_OFFSET = 5
SIZE_OF_INPUT_FIELDS = 6

MAX_WINDOW_SIZE = 2 * 1024

M_CYCLES_PER_SECOND = 4194304.
NORMALIZE_TIME_BY = M_CYCLES_PER_SECOND * 10.

def fresh_input(command, channel, time):
    newd = np.zeros(shape=SIZE_OF_INPUT_FIELDS, dtype=int)
    newd[TIME_OFFSET] = time
    newd[CH_OFFSET] = channel
    newd[CMD_OFFSET] = command
    return newd

def parse_bool(v):
    if v == "true":
        return 1
    elif v == "false":
        return 0
    else:
        return int(v)

def command_of_parts(command, channel, parts, time):
    inp = fresh_input(command, channel, time)
    
    if command == CMD_DUTYLL:
        inp[PARAM1_OFFSET] = int(parts[3])
        inp[PARAM2_OFFSET] = int(parts[4])
    elif command == CMD_VOLENVPER:
        inp[PARAM1_OFFSET] = int(parts[3])
        inp[PARAM2_OFFSET] = parse_bool(parts[4])
        inp[PARAM3_OFFSET] = int(parts[4])
    elif command == CMD_LSB:
        inp[PARAM1_OFFSET] = int(parts[3])
        inp[PARAM2_OFFSET] = 0
        inp[PARAM3_OFFSET] = 0
    elif command == CMD_MSB:
        inp[PARAM1_OFFSET] = int(parts[3])
        inp[PARAM2_OFFSET] = parse_bool(parts[4])
        inp[PARAM3_OFFSET] = parse_bool(parts[5])
    else:
        raise "this should not happen"
    return inp

def int32_as_bytes(ival):
    return np.frombuffer(ival.item().to_bytes(4, byteorder = 'big'), dtype=np.uint8)

def int32_of_bytes(np):
    return int.from_bytes(np, byteorder = 'big')

def int8_as_bytes(ival):
    return np.frombuffer(ival.item().to_bytes(1, byteorder='big'), dtype=np.uint8)

def int8_of_bytes(np):
    return int.from_bytes(np, byteorder = 'big')

def merge_params(data):
    command = data[CMD_OFFSET]
    if command == CMD_DUTYLL:
        return (data[PARAM1_OFFSET] << 6) | data[PARAM2_OFFSET]
    elif command == CMD_VOLENVPER:
        return (data[PARAM1_OFFSET] << 4) | (data[PARAM2_OFFSET] << 3) | data[PARAM3_OFFSET]
    elif command == CMD_LSB:
        return data[PARAM1_OFFSET]
    elif command == CMD_MSB:
        return data[PARAM1_OFFSET]  | (data[PARAM2_OFFSET] << 6) | (data[PARAM3_OFFSET] << 7)
    else:
        raise "this should not happen"
        
def unmerge_params(command, data,v ):
    if command == CMD_DUTYLL:
        data[PARAM1_OFFSET] = v >> 6;
        data[PARAM2_OFFSET] = v & 0b0011_1111
    elif command == CMD_VOLENVPER:
        data[PARAM1_OFFSET] = v >> 4
        data[PARAM2_OFFSET] = (v & 0b0000_1000) >> 3
        data[PARAM3_OFFSET] = (v & 0b0000_0111)
    elif command == CMD_LSB:
        data[PARAM1_OFFSET] = v
    elif command == CMD_MSB:
        data[PARAM1_OFFSET] = v & 0b0011_1111
        data[PARAM2_OFFSET] = (v & 0b0100_0000) >> 6
        data[PARAM3_OFFSET] = (v & 0b1000_0000) >> 7
    else:
        raise Exception("this should not happen")
        
BYTES_PER_ENTRY=7

def command_to_bytes(command):
    new_arr = np.concatenate([
                    int32_as_bytes(command[TIME_OFFSET]),
                    int8_as_bytes(command[CH_OFFSET]),
                    int8_as_bytes(command[CMD_OFFSET]),
                    int8_as_bytes(merge_params(command)),]).flatten()
    return new_arr

def command_of_bytes(byte_arr):
    d = fresh_input(0, 0, 0)
    d[TIME_OFFSET] = int32_of_bytes(byte_arr[0:4])
    print(byte_arr[0:4], d[TIME_OFFSET])
    d[TIME_OFFSET] = min(d[TIME_OFFSET], 400000)
    print("TOFF:", d[TIME_OFFSET])
    print("CH:", byte_arr, byte_arr[4])
    d[CH_OFFSET] = int8_of_bytes(byte_arr[4:5])
    if d[CH_OFFSET] != 1 and d[CH_OFFSET] != 2:
        raise Exception("bad channel prediction")
    d[CMD_OFFSET] = int8_of_bytes(byte_arr[5:6])
    print("NCMD", d[CMD_OFFSET])
    unmerge_params(d[CMD_OFFSET], d, byte_arr[6])
    return d

def print_feature(data, file=sys.stdout):
    
    print("FILE: ", file)
    print("FORPRINT", data)

    command = data[CMD_OFFSET]
    
    if command == CMD_DUTYLL:
        print(f"CH {data[CH_OFFSET]} DUTYLL {data[PARAM1_OFFSET]} {data[PARAM2_OFFSET]} AT {data[TIME_OFFSET]}", file=file, flush=True)
    elif command == CMD_VOLENVPER:
        print(f"CH {data[CH_OFFSET]} VOLENVPER {data[PARAM1_OFFSET]} {data[PARAM2_OFFSET]} {data[PARAM3_OFFSET]} AT {data[TIME_OFFSET]}", file=file, flush=True)
    elif command == CMD_LSB:
        print(f"CH {data[CH_OFFSET]} FREQLSB {data[PARAM1_OFFSET]} AT {data[TIME_OFFSET]}", file=file, flush=True)
    elif command == CMD_MSB:
        print(f"CH {data[CH_OFFSET]} FREQMSB {data[PARAM1_OFFSET]} {data[PARAM2_OFFSET]} {data[PARAM3_OFFSET]} AT {data[TIME_OFFSET]}", file=file, flush=True)
    else:
        print(f"Bad prediction", file=file, flush=True)

def load_training_data(src):
    data = []
    file = open(src, 'r')
    for line in file:
        parts = line.split()
        if len(parts) > 0 and parts[0] == "CH":
            #print(parts)
            channel = int(parts[1])
            command = parts[2]
            time = int(parts[-1])
            if command == "DUTYLL":
                new_item = command_of_parts(CMD_DUTYLL, channel, parts, time)
            elif command == "VOLENVPER":
                new_item = command_of_parts(CMD_VOLENVPER, channel, parts, time)
            elif command == "FREQLSB":
                new_item = command_of_parts(CMD_LSB, channel, parts, time)
            elif command == "FREQMSB":
                new_item = command_of_parts(CMD_MSB, channel, parts, time)
            else:
                print("Unknown", command)
             # Otherwise unknown   
            data.append(new_item)
    return data

@pescador.streamable
def samples_from_training_data(src, window_size):
    sample_data = None
    try:
        sample_data = load_training_data(src)
    except Exception as e:
        LOGGER.error('Could not load {}: {}'.format(src, str(e)))
        raise StopIteration()

    while True:
        if len(sample_data) < window_size:
            sample = sample_data
        else:
            # Sample a random window from the audio file
            start_idx = np.random.randint(0, len(sample_data) - window_size)
            sample = sample_data[start_idx:(start_idx + window_size)]
            
        sample = np.array([command_to_bytes(x) for x in sample])

        yield sample

def create_batch_generator(paths, window_size):
    streamers = []
    
    for path in paths:
        streamers.append(samples_from_training_data(path, window_size))
    
    mux = pescador.StochasticMux(streamers, n_active=1, rate=1).iterate()
    
    return mux

def training_files(dirp):
    return [
      os.path.join(root, fname)
      for (root, dir_names, file_names) in os.walk(dirp, followlinks=True)
      for fname in file_names
    ]

def create_data_split(paths, window_size=MAX_WINDOW_SIZE):
    train_gen = create_batch_generator(paths, window_size)
    return train_gen

class SampleDataset(torch.utils.data.IterableDataset):
    
    def __init__(self, path, window_size):
        super(SampleDataset).__init__()
        self.loader = create_data_split(training_files(path))
    
    def __iter__(self):
        while True:
             yield next(self.loader).flatten()

In [7]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model, dropout = 0.1, max_len = 2048):
        super().__init__()
        
        assert(MAX_WINDOW_SIZE <= max_len)
        
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)

In [8]:
KERNEL_SIZE_SAMPLES=16
KERNEL_SIZE=BYTES_PER_ENTRY * KERNEL_SIZE_SAMPLES

class CausalConv1d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, dilation=1, **kwargs):
        super(CausalConv1d, self).__init__()
        self.pad = (kernel_size - 1) * dilation
        self.conv = nn.Conv1d(in_channels, out_channels, kernel_size , dilation=dilation, **kwargs)
        
    def forward(self, x):
        #pad here to only add to the left side
        x = F.pad(x, (self.pad, 0))
        return self.conv(x)

class ResidualBlock(nn.Module):
    def __init__(self, input_channels, output_channels, kernel_size, skip_channels, dilation=1):
        super(ResidualBlock, self).__init__()
        
        self.conv_sig = CausalConv1d(input_channels, output_channels, kernel_size, dilation)
        self.sig = nn.Sigmoid()
        
        self.conv_tan = CausalConv1d(input_channels, output_channels, kernel_size, dilation)
        self.tanh = nn.Tanh()
        
        #separate weights for residual and skip channels
        self.conv_r = nn.Conv1d(output_channels, output_channels, 1)
        self.conv_s = nn.Conv1d(output_channels, skip_channels, 1)
        
    def forward(self, x):
        o = self.sig(self.conv_sig(x)) * self.tanh(self.conv_tan(x))
        skip = self.conv_s(o)
        residual = self.conv_r(o)
        return residual, skip

# When using dilations the effective lookback is KERNEL_SIZE^num_layers
class CommandNet(nn.Module):
    def __init__(self, skip_channels=256, num_blocks=4, num_layers=5, num_hidden=256, kernel_size=KERNEL_SIZE): 
        super(CommandNet, self).__init__()

        self.embed = nn.Embedding(skip_channels, skip_channels)
        #self.positional_embedding = PositionalEncoding(skip_channels)
        self.causal_conv = CausalConv1d(skip_channels, num_hidden, kernel_size)
        self.res_stack = nn.ModuleList()

        for b in range(num_blocks):
            for i in range(num_layers):
                self.res_stack.append(ResidualBlock(num_hidden, num_hidden, kernel_size, skip_channels=skip_channels, dilation=2**i))
        
        self.relu1 = nn.ReLU()
        self.conv1 = nn.Conv1d(skip_channels, skip_channels, 1)
        self.relu2 = nn.ReLU()
        self.conv2 = nn.Conv1d(skip_channels, skip_channels, 1)
        
    def forward(self, x):

        o = self.embed(x)
        #o = self.positional_embedding(o)
        o = o.permute(0,2,1)
        
        o = self.causal_conv(o)
        
        skip_vals = []
        
        #run res blocks
        for i, layer in enumerate(self.res_stack):
            o, s = layer(o)
            skip_vals.append(s)
            
        #sum skip values and pass to last portion of network
        o = reduce((lambda a,b: a+b), skip_vals)
        
        o = self.relu1(o)
        o = self.conv1(o)
        o = self.relu2(o)
        o = self.conv2(o)
        
        return o

def load(path):
        
    lr = 0.01
    momentum=0.8
    
    command_generator = CommandNet()
    
    optimizer = optim.SGD(
        command_generator.parameters(),
        lr=lr,
        momentum=momentum
    )
    
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.95, min_lr=0.001)
    command_generator = command_generator.to(device)
    
    # This needs to be after to because the optimizer decides what device to send the tensors to based on the
    # device of the model.
    if path != None:
        command_generator.load_state_dict(torch.load(path + ".model"))
        optimizer.load_state_dict(torch.load(path + ".optimizer"))
        #scheduler = torch.load(path + ".scheduler")

    return command_generator, optimizer, scheduler

In [9]:
EPOCHS = 500000
ROUND_SZ = 100

print("Collecting training data")
loader = torch.utils.data.DataLoader(SampleDataset("../../training_data/",window_size=MAX_WINDOW_SIZE))
print("Collected")

def train(path):
    
    command_generator, optimizer, scheduler = load(path)

    criterion = nn.CrossEntropyLoss()
    running_loss = torch.zeros(1, device=device)

    def step():
        
        print("Starting batch")
        running_loss.zero_()
        
        for i in range(ROUND_SZ):
            
            if i % (ROUND_SZ / 10) == 0:
                print("Batch completion:", (float(i) / float(ROUND_SZ)) * 100., "%")
            
            seq = next(iter(loader)).long().to(device)
            inputs = seq[:,:-1]
            labels = seq[:,1:]
    
            optimizer.zero_grad()

            with autocast():
                outputs = command_generator(inputs)
                
                # Backprop only on the datapoints that had at least half a k kernel
                #backprop_l = int(min(KERNEL_SIZE / 2, len(seq) / 2))
                #backprop_inputs = outputs[:,:,backprop_l:]
                #backprop_outputs = labels[:,backprop_l:]
                
                loss = criterion(outputs, labels)

            loss.backward()
            optimizer.step()
            #print(loss.detach().item())
            running_loss.add_(loss.detach())
            #print(running_loss)
            
            seq = seq.detach().to(cpu)
            del inputs
            del labels
            del seq

        result = running_loss / ROUND_SZ
        return result
    
    def save(name):
        torch.save(command_generator.state_dict(), "./" + name + ".model")
        torch.save(optimizer.state_dict(), "./" + name + ".optimizer")
        
        # Saving the scheduler seems to break stuff
        #torch.save(scheduler, "./" + name + ".scheduler")

    for i in range(0, EPOCHS):
        loss = step()
        scheduler.step(loss)

        print("Loss:", loss.item())
        print("LR:", optimizer.param_groups[0]['lr'])
        
        print("Saving checkpoint")
        
        # Timestamp every 10th epoch to test fits later
        if i % 10 == 0:
            save(str(int(datetime.now().timestamp())))

        save("./last.checkpoint")
        print("Saved checkpoint")
    
    return command_generator.eval()

#train(None)
#train("./last.checkpoint")

Collected


In [ ]:
# %%capture cap --no-stderr

print("Collecting training data")
train_gen = create_data_split(training_files("../../out_of_sample/"))
print("Collected")

command_generator, _, _ = load("./last.checkpoint")
command_generator = command_generator.eval()

seed = next(train_gen).flatten()

def max_of(v, begin, end):
    return begin + np.argmax(v[begin:end])

with open('seed.txt', 'w') as f:
    for i in range(0, len(seed), BYTES_PER_ENTRY):
        print("Seed value :", i, seed.shape)
        cmd = command_of_bytes(seed[i:i+BYTES_PER_ENTRY])
        print_feature(cmd)
        print_feature(cmd, file=f)

with open('output.txt', 'w') as f:
    
    for i in range(BYTES_PER_ENTRY * 10000):
        seq = torch.Tensor(seed).long().to(device).unsqueeze(0)
        pred = command_generator(seq).detach().to(cpu).permute(0,2,1).squeeze(0).numpy()
        new_pred_bytes = np.array([np.argmax(x) for x in pred[-1:]]).astype(np.uint8)
        seed = np.concatenate([seed[1:], new_pred_bytes])

        print("Step: ", i, seed.shape)
    
        if (i + 1) % BYTES_PER_ENTRY == 0:
            try:
                print("New pred bytes:", seed[-7:])
                new_pred = command_of_bytes(seed[-7:])
                print_feature(new_pred)
                print_feature(new_pred, file=f)
            except BaseException as err:
                print("pred was not valid because:", err)

    del pred

Collected
Seed value : 0 (14336,)
[ 0  0  0 24] 24
TOFF: 24
CH: [ 0  0  0 24  2  2  4] 2
NCMD 2
FILE:  <ipykernel.iostream.OutStream object at 0x7f4adb75b5e0>
FORPRINT [24  2  2  4  0  0]
CH 2 FREQMSB 4 0 0 AT 24
FILE:  <_io.TextIOWrapper name='seed.txt' mode='w' encoding='UTF-8'>
FORPRINT [24  2  2  4  0  0]
Seed value : 7 (14336,)
[  0   0   0 168] 168
TOFF: 168
CH: [  0   0   0 168   2   1  64] 2
NCMD 1
FILE:  <ipykernel.iostream.OutStream object at 0x7f4adb75b5e0>
FORPRINT [168   2   1   1   0   0]
CH 2 DUTYLL 1 0 AT 168
FILE:  <_io.TextIOWrapper name='seed.txt' mode='w' encoding='UTF-8'>
FORPRINT [168   2   1   1   0   0]
Seed value : 14 (14336,)
[  0   1  10 128] 68224
TOFF: 68224
CH: [  0   1  10 128   1   3  78] 1
NCMD 3
FILE:  <ipykernel.iostream.OutStream object at 0x7f4adb75b5e0>
FORPRINT [68224     1     3    78     0     0]
CH 1 FREQLSB 78 AT 68224
FILE:  <_io.TextIOWrapper name='seed.txt' mode='w' encoding='UTF-8'>
FORPRINT [68224     1     3    78     0     0]
Seed value

FILE:  <_io.TextIOWrapper name='seed.txt' mode='w' encoding='UTF-8'>
FORPRINT [160   1   1   0   0   0]
Seed value : 182 (14336,)
[0 0 5 0] 1280
TOFF: 1280
CH: [ 0  0  5  0  2  3 78] 2
NCMD 3
FILE:  <ipykernel.iostream.OutStream object at 0x7f4adb75b5e0>
FORPRINT [1280    2    3   78    0    0]
CH 2 FREQLSB 78 AT 1280
FILE:  <_io.TextIOWrapper name='seed.txt' mode='w' encoding='UTF-8'>
FORPRINT [1280    2    3   78    0    0]
Seed value : 189 (14336,)
[ 0  0  0 24] 24
TOFF: 24
CH: [ 0  0  0 24  2  2  4] 2
NCMD 2
FILE:  <ipykernel.iostream.OutStream object at 0x7f4adb75b5e0>
FORPRINT [24  2  2  4  0  0]
CH 2 FREQMSB 4 0 0 AT 24
FILE:  <_io.TextIOWrapper name='seed.txt' mode='w' encoding='UTF-8'>
FORPRINT [24  2  2  4  0  0]
Seed value : 196 (14336,)
[  0   0   0 168] 168
TOFF: 168
CH: [  0   0   0 168   2   1  64] 2
NCMD 1
FILE:  <ipykernel.iostream.OutStream object at 0x7f4adb75b5e0>
FORPRINT [168   2   1   1   0   0]
CH 2 DUTYLL 1 0 AT 168
FILE:  <_io.TextIOWrapper name='seed.txt' mod

FILE:  <_io.TextIOWrapper name='seed.txt' mode='w' encoding='UTF-8'>
FORPRINT [28  1  2  6  0  1]
Seed value : 364 (14336,)
[ 0  0  0 16] 16
TOFF: 16
CH: [ 0  0  0 16  1  2  6] 1
NCMD 2
FILE:  <ipykernel.iostream.OutStream object at 0x7f4adb75b5e0>
FORPRINT [16  1  2  6  0  0]
CH 1 FREQMSB 6 0 0 AT 16
FILE:  <_io.TextIOWrapper name='seed.txt' mode='w' encoding='UTF-8'>
FORPRINT [16  1  2  6  0  0]
Seed value : 371 (14336,)
[  0   0   4 216] 1240
TOFF: 1240
CH: [  0   0   4 216   2   3  78] 2
NCMD 3
FILE:  <ipykernel.iostream.OutStream object at 0x7f4adb75b5e0>
FORPRINT [1240    2    3   78    0    0]
CH 2 FREQLSB 78 AT 1240
FILE:  <_io.TextIOWrapper name='seed.txt' mode='w' encoding='UTF-8'>
FORPRINT [1240    2    3   78    0    0]
Seed value : 378 (14336,)
[ 0  0  0 24] 24
TOFF: 24
CH: [ 0  0  0 24  2  2  4] 2
NCMD 2
FILE:  <ipykernel.iostream.OutStream object at 0x7f4adb75b5e0>
FORPRINT [24  2  2  4  0  0]
CH 2 FREQMSB 4 0 0 AT 24
FILE:  <_io.TextIOWrapper name='seed.txt' mode='w' en

FILE:  <_io.TextIOWrapper name='seed.txt' mode='w' encoding='UTF-8'>
FORPRINT [1280    2    3  138    0    0]
Seed value : 546 (14336,)
[ 0  0  0 24] 24
TOFF: 24
CH: [ 0  0  0 24  2  2  7] 2
NCMD 2
FILE:  <ipykernel.iostream.OutStream object at 0x7f4adb75b5e0>
FORPRINT [24  2  2  7  0  0]
CH 2 FREQMSB 7 0 0 AT 24
FILE:  <_io.TextIOWrapper name='seed.txt' mode='w' encoding='UTF-8'>
FORPRINT [24  2  2  7  0  0]
Seed value : 553 (14336,)
[  0   0   0 168] 168
TOFF: 168
CH: [  0   0   0 168   2   1  64] 2
NCMD 1
FILE:  <ipykernel.iostream.OutStream object at 0x7f4adb75b5e0>
FORPRINT [168   2   1   1   0   0]
CH 2 DUTYLL 1 0 AT 168
FILE:  <_io.TextIOWrapper name='seed.txt' mode='w' encoding='UTF-8'>
FORPRINT [168   2   1   1   0   0]
Seed value : 560 (14336,)
[  0   1  10 152] 68248
TOFF: 68248
CH: [  0   1  10 152   1   3 156] 1
NCMD 3
FILE:  <ipykernel.iostream.OutStream object at 0x7f4adb75b5e0>
FORPRINT [68248     1     3   156     0     0]
CH 1 FREQLSB 156 AT 68248
FILE:  <_io.TextIOWr

FILE:  <_io.TextIOWrapper name='seed.txt' mode='w' encoding='UTF-8'>
FORPRINT [168   2   1   1   0   0]
Seed value : 728 (14336,)
[  0   0 255 248] 65528
TOFF: 65528
CH: [  0   0 255 248   1   3  39] 1
NCMD 3
FILE:  <ipykernel.iostream.OutStream object at 0x7f4adb75b5e0>
FORPRINT [65528     1     3    39     0     0]
CH 1 FREQLSB 39 AT 65528
FILE:  <_io.TextIOWrapper name='seed.txt' mode='w' encoding='UTF-8'>
FORPRINT [65528     1     3    39     0     0]
Seed value : 735 (14336,)
[ 0  0  0 24] 24
TOFF: 24
CH: [ 0  0  0 24  1  2  6] 1
NCMD 2
FILE:  <ipykernel.iostream.OutStream object at 0x7f4adb75b5e0>
FORPRINT [24  1  2  6  0  0]
CH 1 FREQMSB 6 0 0 AT 24
FILE:  <_io.TextIOWrapper name='seed.txt' mode='w' encoding='UTF-8'>
FORPRINT [24  1  2  6  0  0]
Seed value : 742 (14336,)
[  0   0   0 160] 160
TOFF: 160
CH: [  0   0   0 160   1   1   0] 1
NCMD 1
FILE:  <ipykernel.iostream.OutStream object at 0x7f4adb75b5e0>
FORPRINT [160   1   1   0   0   0]
CH 1 DUTYLL 0 0 AT 160
FILE:  <_io.Tex

...

FILE:  <_io.TextIOWrapper name='output.txt' mode='w' encoding='UTF-8'>
FORPRINT [24  2  3  6  0  0]
Step:  1456 (14336,)
Step:  1457 (14336,)
Step:  1458 (14336,)
Step:  1459 (14336,)
Step:  1460 (14336,)
Step:  1461 (14336,)
Step:  1462 (14336,)
New pred bytes: [  0   0   0  24   2   2 130]
[ 0  0  0 24] 24
TOFF: 24
CH: [  0   0   0  24   2   2 130] 2
NCMD 2
FILE:  <ipykernel.iostream.OutStream object at 0x7f4adb75b5e0>
FORPRINT [24  2  2  2  0  1]
CH 2 FREQMSB 2 0 1 AT 24
FILE:  <_io.TextIOWrapper name='output.txt' mode='w' encoding='UTF-8'>
FORPRINT [24  2  2  2  0  1]
Step:  1463 (14336,)
Step:  1464 (14336,)
Step:  1465 (14336,)
Step:  1466 (14336,)
Step:  1467 (14336,)
Step:  1468 (14336,)
Step:  1469 (14336,)
New pred bytes: [  0   1  21 112   2   1 128]
[  0   1  21 112] 71024
TOFF: 71024
CH: [  0   1  21 112   2   1 128] 2
NCMD 1
FILE:  <ipykernel.iostream.OutStream object at 0x7f4adb75b5e0>
FORPRINT [71024     2     1     2     0     0]
CH 2 DUTYLL 2 0 AT 71024
FILE:  <_io.Te

FILE:  <_io.TextIOWrapper name='output.txt' mode='w' encoding='UTF-8'>
FORPRINT [64  2  0  8  0  0]
Step:  1575 (14336,)
Step:  1576 (14336,)
Step:  1577 (14336,)
Step:  1578 (14336,)
Step:  1579 (14336,)
Step:  1580 (14336,)
Step:  1581 (14336,)
New pred bytes: [ 0  0  0 24  2  3 99]
[ 0  0  0 24] 24
TOFF: 24
CH: [ 0  0  0 24  2  3 99] 2
NCMD 3
FILE:  <ipykernel.iostream.OutStream object at 0x7f4adb75b5e0>
FORPRINT [24  2  3 99  0  0]
CH 2 FREQLSB 99 AT 24
FILE:  <_io.TextIOWrapper name='output.txt' mode='w' encoding='UTF-8'>
FORPRINT [24  2  3 99  0  0]
Step:  1582 (14336,)
Step:  1583 (14336,)
Step:  1584 (14336,)
Step:  1585 (14336,)
Step:  1586 (14336,)
Step:  1587 (14336,)
Step:  1588 (14336,)
New pred bytes: [  0   0   0  24   2   2 133]
[ 0  0  0 24] 24
TOFF: 24
CH: [  0   0   0  24   2   2 133] 2
NCMD 2
FILE:  <ipykernel.iostream.OutStream object at 0x7f4adb75b5e0>
FORPRINT [24  2  2  5  0  1]
CH 2 FREQMSB 5 0 1 AT 24
FILE:  <_io.TextIOWrapper name='output.txt' mode='w' encodi

Step:  1694 (14336,)
Step:  1695 (14336,)
Step:  1696 (14336,)
Step:  1697 (14336,)
Step:  1698 (14336,)
Step:  1699 (14336,)
Step:  1700 (14336,)
New pred bytes: [ 0  0  0 24  1  1 64]
[ 0  0  0 24] 24
TOFF: 24
CH: [ 0  0  0 24  1  1 64] 1
NCMD 1
FILE:  <ipykernel.iostream.OutStream object at 0x7f4adb75b5e0>
FORPRINT [24  1  1  1  0  0]
CH 1 DUTYLL 1 0 AT 24
FILE:  <_io.TextIOWrapper name='output.txt' mode='w' encoding='UTF-8'>
FORPRINT [24  1  1  1  0  0]
Step:  1701 (14336,)
Step:  1702 (14336,)
Step:  1703 (14336,)
Step:  1704 (14336,)
Step:  1705 (14336,)
Step:  1706 (14336,)
Step:  1707 (14336,)
New pred bytes: [  0   0   0  64   1   0 128]
[ 0  0  0 64] 64
TOFF: 64
CH: [  0   0   0  64   1   0 128] 1
NCMD 0
FILE:  <ipykernel.iostream.OutStream object at 0x7f4adb75b5e0>
FORPRINT [64  1  0  8  0  0]
CH 1 VOLENVPER 8 0 0 AT 64
FILE:  <_io.TextIOWrapper name='output.txt' mode='w' encoding='UTF-8'>
FORPRINT [64  1  0  8  0  0]
Step:  1708 (14336,)
Step:  1709 (14336,)
Step:  1710 (14

Step:  1819 (14336,)
New pred bytes: [ 0  0  0 64  1  0 80]
[ 0  0  0 64] 64
TOFF: 64
CH: [ 0  0  0 64  1  0 80] 1
NCMD 0
FILE:  <ipykernel.iostream.OutStream object at 0x7f4adb75b5e0>
FORPRINT [64  1  0  5  0  0]
CH 1 VOLENVPER 5 0 0 AT 64
FILE:  <_io.TextIOWrapper name='output.txt' mode='w' encoding='UTF-8'>
FORPRINT [64  1  0  5  0  0]
Step:  1820 (14336,)
Step:  1821 (14336,)
Step:  1822 (14336,)
Step:  1823 (14336,)
Step:  1824 (14336,)
Step:  1825 (14336,)
Step:  1826 (14336,)
New pred bytes: [ 0  0  0 24  1  3 34]
[ 0  0  0 24] 24
TOFF: 24
CH: [ 0  0  0 24  1  3 34] 1
NCMD 3
FILE:  <ipykernel.iostream.OutStream object at 0x7f4adb75b5e0>
FORPRINT [24  1  3 34  0  0]
CH 1 FREQLSB 34 AT 24
FILE:  <_io.TextIOWrapper name='output.txt' mode='w' encoding='UTF-8'>
FORPRINT [24  1  3 34  0  0]
Step:  1827 (14336,)
Step:  1828 (14336,)
Step:  1829 (14336,)
Step:  1830 (14336,)
Step:  1831 (14336,)
Step:  1832 (14336,)
Step:  1833 (14336,)
New pred bytes: [  0   0   0  24   1   2 128]
[ 0 

FILE:  <_io.TextIOWrapper name='output.txt' mode='w' encoding='UTF-8'>
FORPRINT [24  1  1  2  0  0]
Step:  1939 (14336,)
Step:  1940 (14336,)
Step:  1941 (14336,)
Step:  1942 (14336,)
Step:  1943 (14336,)
Step:  1944 (14336,)
Step:  1945 (14336,)
New pred bytes: [  0   0   0  64   1   0 176]
[ 0  0  0 64] 64
TOFF: 64
CH: [  0   0   0  64   1   0 176] 1
NCMD 0
FILE:  <ipykernel.iostream.OutStream object at 0x7f4adb75b5e0>
FORPRINT [64  1  0 11  0  0]
CH 1 VOLENVPER 11 0 0 AT 64
FILE:  <_io.TextIOWrapper name='output.txt' mode='w' encoding='UTF-8'>
FORPRINT [64  1  0 11  0  0]
Step:  1946 (14336,)
Step:  1947 (14336,)
Step:  1948 (14336,)
Step:  1949 (14336,)
Step:  1950 (14336,)
Step:  1951 (14336,)
Step:  1952 (14336,)
New pred bytes: [  0   0   0  24   1   3 131]
[ 0  0  0 24] 24
TOFF: 24
CH: [  0   0   0  24   1   3 131] 1
NCMD 3
FILE:  <ipykernel.iostream.OutStream object at 0x7f4adb75b5e0>
FORPRINT [ 24   1   3 131   0   0]
CH 1 FREQLSB 131 AT 24
FILE:  <_io.TextIOWrapper name='out